# Get All The Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
from IPython.display import display, Markdown

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Import utils with different aliases
from src.utils import csv_exporter as csv_utils
from src.utils import validation as val_utils
from src.utils import transformations as trans_utils
from src.utils import data_merger as merge_utils
from src.utils import config_validator as config_utils
from src.utils import metrics as metric_utils

# Read the CSV files using our utility function and alining start dates
data_dir = project_root / 'data'
sprds = csv_utils.read_csv_to_df(data_dir / 'sprds_data.csv', fill='ffill', start_date_align='yes')
derv = csv_utils.read_csv_to_df(data_dir / 'derv_data.csv', fill='ffill', start_date_align='yes')
er_ytd = csv_utils.read_csv_to_df(data_dir / 'er_ytd_data.csv', fill='ffill', start_date_align='yes')

# Convert er_ytd table into credit unlevered indices
credit_unlevered_index = trans_utils.convert_er_ytd_to_index(er_ytd)
credit_unlevered_index = credit_unlevered_index[['cad_ig_oas_index', 'us_ig_oas_index','us_hy_oas_index']]
# Create levered index using get_er_index
credit_levered_index = trans_utils.get_er_index(credit_unlevered_index, cost_of_borrow=40, leverage=3.0)

# Get just the columns i want from sprds
sprds_selected = sprds[['cad_ig_oas', 'us_ig_oas', 'us_hy_oas']]

# Merge sprds with credit_unlevered_index
sprds_credit_unlevered_index = merge_utils.merge_dfs(sprds_selected, credit_unlevered_index, fill='ffill', start_date_align='yes')
sprds_credit_unlevered_index.info()

2024-12-09 13:43:59,143 - WARNING - Cannot infer frequency. Attempting to resample to daily frequency.
2024-12-09 13:43:59,145 - INFO - Inferred frequency: D


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5507 entries, 2002-12-31 to 2024-12-06
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cad_ig_oas        5507 non-null   float64
 1   us_ig_oas         5507 non-null   float64
 2   us_hy_oas         5507 non-null   float64
 3   cad_ig_oas_index  5507 non-null   float64
 4   us_ig_oas_index   5507 non-null   float64
 5   us_hy_oas_index   5507 non-null   float64
dtypes: float64(6)
memory usage: 301.2 KB


# Constructing The Spread Table

In [31]:
import pandas as pd
import numpy as np
from IPython.display import display

# get just the columns i want from sprds
sprds_abs_marketing_table = sprds[['cad_ig_oas', 'us_ig_oas', 'us_hy_oas', 
       'cad_ig_oas_1-5yr', 'cad_ig_oas_5-10yr', 'cad_ig_oas_>10yr',
       'cad_ig_oas_fins', 'cad_ig_oas_industrials', 'cad_ig_oas_utility',
       'cad_ig_oas_a', 'cad_ig_oas_bbb', 'cad_prov_oas', 'cad_prov_oas_1-5yr',
       'cad_prov_oas_5-10yr', 'cad_prov_oas_>10yr', 'cad_prov_cmb',
       'us_ig_oas_a', 'us_ig_oas_bbb',
       'us_ig_oas_1-3yr', 'us_ig_oas_3-5yr', 'us_ig_oas_5-7yr',
       'us_ig_oas_7-10yr', 'us_ig_oas_>10yr', 'us_ig_oas_industrials',
       'us_ig_oas_uts', 'us_ig_oas_fins']]

def create_spreads_summary_table(df: pd.DataFrame, 
                               start_date: str = None,
                               end_date: str = None,
                               include_markets: list = None,
                               exclude_markets: list = None) -> pd.DataFrame:
    """
    Create a formatted summary table of spread metrics.
    
    Args:
        df (pd.DataFrame): DataFrame containing spread series with datetime index
        start_date (str, optional): Start date for analysis in 'YYYY-MM-DD' format
        end_date (str, optional): End date for analysis in 'YYYY-MM-DD' format
        include_markets (list, optional): List of market names to include
        exclude_markets (list, optional): List of market names to exclude
        
    Returns:
        pd.DataFrame: Formatted summary table
    """
    # Create a copy of the DataFrame to avoid modifying the original
    df = df.copy()
    
    # Filter date range if specified
    if start_date:
        df = df[df.index >= pd.to_datetime(start_date)]
    if end_date:
        df = df[df.index <= pd.to_datetime(end_date)]
    
    # Filter markets if specified
    if include_markets:
        df = df[include_markets]
    if exclude_markets:
        df = df[[col for col in df.columns if col not in exclude_markets]]
    
    # Get current date and relevant dates for changes
    current_date = df.index[-1]
    week_ago = current_date - pd.Timedelta(days=7)
    month_start = current_date.replace(day=1)
    quarter_start = pd.Timestamp(f"{current_date.year}-{((current_date.month-1)//3)*3 + 1}-01")
    year_start = pd.Timestamp(f"{current_date.year}-01-01")
    year_ago = current_date - pd.Timedelta(days=365)
    
    # Create empty results dictionary
    results = []
    
    for column in df.columns:
        series = df[column]
        
        # Calculate metrics
        last_value = series.iloc[-1]
        min_value = series.min()
        max_value = series.max()
        
        # Get dates of extremes
        min_date = series[series == min_value].index[0]
        max_date = series[series == max_value].index[0]
        
        # Calculate changes with error handling
        try:
            wow_change = last_value - series[series.index <= week_ago].iloc[-1]
        except IndexError:
            wow_change = np.nan
            
        try:
            mtd_change = last_value - series[series.index <= month_start].iloc[-1]
        except IndexError:
            mtd_change = np.nan
            
        try:
            qtd_change = last_value - series[series.index <= quarter_start].iloc[-1]
        except IndexError:
            qtd_change = np.nan
            
        try:
            ytd_change = last_value - series[series.index <= year_start].iloc[-1]
        except IndexError:
            ytd_change = np.nan
            
        try:
            yoy_change = last_value - series[series.index <= year_ago].iloc[-1]
        except IndexError:
            yoy_change = np.nan
        
        # Calculate 2021 minimum
        mask_2021 = (series.index.year == 2021)
        min_2021 = series[mask_2021].min() if any(mask_2021) else np.nan
        
        # Calculate percentile rank using pandas
        clean_series = series.dropna()
        percentile = (clean_series.rank(pct=True).iloc[-1] * 100)
        
        results.append({
            'Market': column,
            'Start Date': series.index[0].strftime('%m/%d/%Y'),
            'End Date': series.index[-1].strftime('%m/%d/%Y'),
            'Last': last_value,
            'WoW Chg': wow_change,
            'MTD Chg': mtd_change,
            'QTD Chg': qtd_change,
            'YTD Chg': ytd_change,
            '1yr Chg': yoy_change,
            'vs Min': last_value - min_value,
            'vs Max': last_value - max_value,
            'Date of Tights': min_date.strftime('%m/%d/%Y'),
            'Date of Wides': max_date.strftime('%m/%d/%Y'),
            'Percentile': percentile,
            'vs 2021 Tights': last_value - min_2021
        })
    
  # Create DataFrame from results and sort by Percentile
    summary_df = pd.DataFrame(results)
    summary_df = summary_df.sort_values('Percentile', ascending=False)
    
    # Set Market as index
    summary_df.set_index('Market', inplace=True)
    
    # Define numeric columns
    numeric_cols = ['Last', 'WoW Chg', 'MTD Chg', 'QTD Chg', 'YTD Chg', '1yr Chg', 
                   'vs Min', 'vs Max', 'vs 2021 Tights']
    
    # Create formatter dictionary for all columns
    formatters = {
        'Percentile': lambda x: f'{x:.1f}%',
        **{col: lambda x: f'{float(x):.1f}' if pd.notnull(x) else '' for col in numeric_cols}
    }
    
   # Apply styling with formatters
    styled_df = summary_df.style.format(formatters).set_properties(**{
        'text-align': 'center'
    }).set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center')]},
        {'selector': '', 'props': [('border', '1px solid black')]},
        {'selector': 'th', 'props': [('font-weight', 'bold')]},
        {'selector': 'caption', 'props': [('caption-side', 'top'), 
                                        ('font-size', '16px'),
                                        ('font-weight', 'bold'),
                                        ('text-align', 'left')]},
        # Add footnote style at the bottom
        {'selector': 'tbody:after', 'props': [
            ('content', '"Source: Bloomberg"'),
            ('display', 'block'),
            ('font-size', '12px'),
            ('font-style', 'italic'),
            ('text-align', 'left'),
            ('padding-top', '8px')
        ]}
    ]).set_caption('Credit Market Spread Analysis')
    
    return styled_df

# Example usage with all data (default):
spread_summary = create_spreads_summary_table(
    sprds_abs_marketing_table)
display(spread_summary)

# Example usage with date range (optional):
# spread_summary_dated = create_spreads_summary_table(
#     sprds_abs_marketing_table,
#     start_date='2023-01-01',
#     end_date='2023-12-31',
#     include_markets=['cad_ig_oas', 'us_ig_oas', 'us_hy_oas']
# )
# display(spread_summary_dated)

,Start Date,End Date,Last,WoW Chg,MTD Chg,QTD Chg,YTD Chg,1yr Chg,vs Min,vs Max,Date of Tights,Date of Wides,Percentile,vs 2021 Tights
Market,,,,,,,,,,,,,,
cad_prov_oas_>10yr,12/31/2002,12/06/2024,87.9,0.5,0.5,-10.3,-3.3,-5.5,62.9,-58.0,01/30/2004,02/27/2009,51.9%,13.6
cad_prov_cmb,12/31/2002,12/06/2024,64.4,0.1,0.1,-8.2,-3.8,-4.9,51.4,-81.9,01/09/2004,12/17/2008,37.5%,6.8
cad_prov_oas,12/31/2002,12/06/2024,61.9,-0.0,-0.0,-8.2,-3.8,-4.8,48.1,-67.9,01/30/2004,02/27/2009,36.2%,6.9
cad_ig_oas_utility,12/31/2002,12/06/2024,110.3,-1.6,-1.6,-19.8,-28.9,-32.9,89.2,-182.8,01/07/2003,01/02/2009,27.9%,-4.1
cad_ig_oas_1-5yr,12/31/2002,12/06/2024,74.8,-0.7,-0.7,-14.4,-33.5,-42.3,38.3,-279.5,02/28/2005,12/31/2008,26.2%,16.8
cad_ig_oas_5-10yr,12/31/2002,12/06/2024,113.2,-3.0,-3.0,-20.8,-39.9,-49.3,73.8,-349.6,04/30/2004,12/31/2008,25.9%,7.9
cad_ig_oas_fins,12/31/2002,12/06/2024,82.5,-0.6,-0.6,-13.2,-38.0,-47.8,56.7,-330.4,01/09/2004,01/02/2009,25.1%,10.6
cad_ig_oas_>10yr,12/31/2002,12/06/2024,152.7,-0.1,-0.1,-18.2,-22.9,-27.2,95.7,-211.9,12/31/2002,12/31/2008,23.7%,-8.9
cad_ig_oas,12/31/2002,12/06/2024,100.2,-1.0,-1.0,-16.6,-31.6,-39.4,49.5,-306.8,08/19/2005,01/02/2009,23.2%,2.0
